In [ ]:
import string

In [ ]:
letters = string.ascii_lowercase

A transition is represented as a dictionary with these keys:
- `curr_state`
- `curr_val1`
- `curr_val2`
- `next_state`
- `next_val1`
- `next_val2`
- `head1move`
- `head2move`

Function for printing a transition in the format needed by [turingmachinesimulator.com](https://turingmachinesimulator.com/):

In [ ]:
def pp_transition(trans):

    return (
        "%s,%s,%s\n%s,%s,%s,%s,%s\n\n"
        % (
            trans['curr_state'],
            trans['curr_val1'],
            trans['curr_val2'],
            trans['next_state'],
            trans['next_val1'],
            trans['next_val2'],
            trans['head1move'],
            trans['head2move']
        )
    )

Convenience function; another way of making transitions.

In [ ]:
def mk_trans(curr_state, curr_val1, curr_val2, next_state, next_val1, next_val2, head1move, head2move):
    return {
        'curr_state': curr_state,
        'curr_val1': curr_val1,
        'curr_val2': curr_val2,
        'next_state': next_state,
        'next_val1': next_val1,
        'next_val2': next_val2,
        'head1move': head1move,
        'head2move': head2move
    }

**We build up the Turing Machine spec as a list of transitions.**

In [ ]:
prog = []

**State q0:** Working forwards through a word cutting out letters; at an odd position

In [ ]:
# If you see a letter, copy it to tape 2, replace with special marker "/"

for x in letters:
    prog.append({
        'curr_state': "q0",
        'curr_val1': x,
        'curr_val2': "_",
        'next_state': "q1",
        'next_val1': "/",
        'next_val2': x,
        'head1move': ">",
        'head2move': "<"
    })

**State q1:** Working forwards through a word cutting out letters, at an even position

In [ ]:
# Leave letters alone, just jog on

for x in letters:
    prog.append({
        'curr_state': "q1",
        'curr_val1': x,
        'curr_val2': "_",
        'next_state': "q0",
        'next_val1': x,
        'next_val2': "/",
        'head1move': ">",
        'head2move': "<"
    })

Recognising the end of the word in q0 and q1:

In [ ]:
# If you reach a space or the end-of-sentence fullstop while in q0 or q1, then move on to q2.
# Move tape 1 head left so we're back into the word.
# Leave tape head 2 pointing to its space.

prog.append(mk_trans("q0", "_", "_",
                     "q2", "_", "_", "<", "-"))

prog.append(mk_trans("q1", "_" ,"_",
                     "q2", "_" ,"_", "<" ,"-"))

prog.append(mk_trans("q0", ".", "_",
                     "q2", ".", "_", "<" , "-"))

prog.append(mk_trans("q1", ".", "_",
                     "q2", ".", "_", "<", "-"))


**State q2:** Run tape 1 head left to get back to the start of the word.

In [ ]:
# If you see a letter, keep going left.

for x in letters:
    prog.append({
        'curr_state': "q2",
        'curr_val1': x,
        'curr_val2': "_",
        'next_state': "q2",
        'next_val1': x,
        'next_val2': "_",
        'head1move': "<",
        'head2move': "-"
    })

# If you see the / symbol, keep going left.

for x in letters:
    prog.append({
        'curr_state': "q2",
        'curr_val1': "/",
        'curr_val2': "_",
        'next_state': "q2",
        'next_val1': "/",
        'next_val2': "_",
        'head1move': "<",
        'head2move': "-"
    })

# If you read a space, move onto q3.
# Shift tape 1 head right, back into the word.
# Also shift tape 2 head right, back into the stored letters.

for x in letters:
    prog.append({
        'curr_state': "q2",
        'curr_val1': "_",
        'curr_val2': "_",
        'next_state': "q3",
        'next_val1': "_",
        'next_val2': "_",
        'head1move': ">",
        'head2move': ">"
    })


**State q3:** We're running forward through the word, putting the stored letters back (in a different order).

In [ ]:
# If we're pointing to a letter that is being left undisturbed

for x in letters:
    prog.append({
        'curr_state': "q3",
        'curr_val1': x,
        'curr_val2': "/",
        'next_state': "q3",
        'next_val1': x,
        'next_val2': "_",
        'head1move': ">",
        'head2move': ">"
    })

# If we're pointing to a "/" where we have to put back a letter.

for x in letters:
    prog.append({
        'curr_state': "q3",
        'curr_val1': "/",
        'curr_val2': x,
        'next_state': "q3",
        'next_val1': x,
        'next_val2': "_",
        'head1move': ">",
        'head2move': ">"
    })

# If we're pointing at a "/" on both heads. This will happen at the start of even-length words.

prog.append({
    'curr_state': "q3",
    'curr_val1': "/",
    'curr_val2': "/",
    'next_state': "q3",
    'next_val1': "/",
    'next_val2': "_",
    'head1move': "-",
    'head2move': ">"
})

# If we're pointing to a letter on tape 1 but a space on tape 2. This will happen at the end of even-length words.

for x in letters:
    prog.append({
        'curr_state': "q3",
        'curr_val1': x,
        'curr_val2': "_",
        'next_state': "q3",
        'next_val1': x,
        'next_val2': "_",
        'head1move': ">",
        'head2move': "-"
    })
    
# If we're pointing at a space on both heads then skip forward and go to q0 to start on the next word.

prog.append({
    'curr_state': "q3",
    'curr_val1': "_",
    'curr_val2': "_",
    'next_state': "q0",
    'next_val1': "_",
    'next_val2': "_",
    'head1move': ">",
    'head2move': "-"
})

# If we're pointing at a fullstop then we're done!

prog.append({
    'curr_state': "q3",
    'curr_val1': ".",
    'curr_val2': "_",
    'next_state': "qdone",
    'next_val1': ".",
    'next_val2': "_",
    'head1move': "-",
    'head2move': "-"
})


**Now print the program in the required syntax for the simulator.**

In [ ]:
s = """
name: Brandwatch Holiday Puzzler
init: q0
accept: qdone

"""

In [ ]:
for transition in prog:
    s = s + pp_transition(transition)

In [ ]:
print(s)